In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.callbacks import Callback


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Dense, LeakyReLU, Embedding, Concatenate, Reshape, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def _discriminator(input_shape=(28, 28, 1), n_classes=10):
    # 1. Khởi tạo nhánh input là y_label
    y_label = Input(shape=(1,))
    # Embedding y_label và chiếu lên không gian véc tơ 50 dimension.
    y_embedding = Embedding(n_classes, 50)(y_label)
    # Gia tăng kích thước y_embedding thông qua linear projection
    n_shape = input_shape[0] * input_shape[1]
    li = Dense(n_shape)(y_embedding)
    li = Reshape((input_shape[0], input_shape[1], 1))(li)

    # 2. Khởi tạo nhánh input là image
    inpt_image = Input(shape=input_shape)

    # 3. Concate y_label và image
    concat = Concatenate()([inpt_image, li])
    # 4. Feature extractor thông qua CNN blocks:
    fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(concat)
    fe = LeakyReLU(alpha=0.2)(fe)

    fe = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)

    # Flatten output
    fe = Flatten()(fe)
    fe = Dropout(0.4)(fe)
    out_layer = Dense(1, activation='sigmoid')(fe)

    # Khởi tạo model
    model = Model([inpt_image, y_label], out_layer)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
def _generator(latent_dim=100, n_classes=10):
  # 1. Khởi tạo nhánh đầu vào là y_label
  y_label = Input(shape=(1,))
  # embedding véc tơ categorical đầu vào
  li = Embedding(n_classes, 50)(y_label)
  n_shape = 7 * 7
  li = Dense(n_shape)(li)
  # reshape lại đầu vào về kích thước 7x7x1 như một channel bổ sung.
  li = Reshape((7, 7, 1))(li)

  # 2. Khởi tạo nhánh đầu vào là véc tơ noise x
  in_lat = Input(shape=(latent_dim,))
  n_shape = 128 * 7 * 7
  gen = Dense(n_shape)(in_lat)
  gen = LeakyReLU(alpha=0.2)(gen)
  # Biến đổi về kích thước 7x7x128
  gen = Reshape((7, 7, 128))(gen)

  # 3. Merge nhánh 1 và nhánh 2
  merge = Concatenate()([gen, li])

  # 4. Sử dụng Conv2DTranspose để giải chập về kích thước ban đầu.
  gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
  gen = LeakyReLU(alpha=0.2)(gen)

  gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
  gen = LeakyReLU(alpha=0.2)(gen)
  # output
  out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
  # model
  model = Model([in_lat, y_label], out_layer)
  return model

In [ ]:
def _cGAN(g_model, d_model):
	# Do cGAN được sử dụng để huấn luyện generator nên discriminator sẽ được đóng băng
	d_model.trainable = False
	# Lấy đầu vào của generator model bao gồm véc tơ noise và nhãn
	gen_noise, gen_label = g_model.input
	# Lấy ảnh sinh ra từ generator model
	gen_output = g_model.output
	# Truyền output và nhãn của mô hình generator vào mô hình discriminator
	gan_output = d_model([gen_output, gen_label])
	# Khởi tạo mô hình cGAN
	model = Model([gen_noise, gen_label], gan_output)
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
# Hàm chuẩn hóa dữ liệu huấn luyện
def _standardize_data(X_train, y_train):
	X = np.expand_dims(X_train, axis=-1)
	X = X.astype('float32')
  # chuẩn hóa dữ liệu về khoảng [-1, 1]
	X = (X - 127.5) / 127.5
	return [X, y_train]

In [ ]:
# Sinh ra các véc tơ noise trong không gian latent space làm đầu vào cho generator
def _generate_latent_points(latent_dim, n_samples, n_classes=10):
	# Khởi tạo các points trong latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape thành batch để feed vào generator.
	z_input = x_input.reshape(n_samples, latent_dim)
	# khởi tạo labels một cách ngẫu nhiên.
	labels = np.random.randint(0, n_classes, n_samples)
	return [z_input, labels]

In [ ]:
# Lựa chọn ngẫu nhiên các dữ liệu huấn luyện
def _generate_real_samples(dataset, n_samples):
    images, labels = dataset
    indices = tf.random.uniform(shape=(n_samples,), minval=0, maxval=images.shape[0], dtype=tf.int32)
    X, labels = tf.gather(images, indices), tf.gather(labels, indices)
    y = tf.ones((n_samples, 1))
    return [X, labels], y

In [ ]:
# Sử dụng generator để sinh ra n_samples ảnh fake.
def _generate_fake_samples(generator, latent_dim, n_samples):
    z_input, labels_input = _generate_latent_points(latent_dim, n_samples)
    z_input = tf.constant(z_input)
    labels_input = tf.constant(labels_input)
    images = generator([z_input, labels_input], training=False)
    y = tf.zeros((n_samples, 1))
    return [images, labels_input], y

In [ ]:
def on_epoch_end(generator, num_img, latent_dim, epoch):
    z_input, labels_input = _generate_latent_points(latent_dim, num_img)
    z_input = tf.constant(z_input)
    labels_input = tf.constant(labels_input)
    generated_images = generator([z_input, labels_input], training=False)
    generated_images = (generated_images * 127.5) + 127.5

    for i in range(num_img):
        img = generated_images[i].numpy()
        img = tf.keras.preprocessing.image.array_to_img(img)
        img.save("generated_img_{labels_input}_{i}_{epoch}.png".format(i=i, epoch=epoch, labels_input=labels_input[i]))

In [ ]:
def _train( g_model, d_model, cGAN_model, dataset, latent_dim, n_epochs=100, n_batch=128, save_every_epochs=10):
    '''
    g_model: generator model
    d_model: discriminator model
    cGAN_model: gan_model
    dataset: dữ liệu huấn luyện, bao gồm: (X_train, y_train)
    latent_dim: Số chiều của latent space
    n_epochs: Số lượng epochs
    n_batch: Kích thước batch_size
    save_every_epochs: Số lượng epochs mà chúng ta sẽ save model.
    '''
    # Tính số lượng batch trên một epochs
    batch_per_epoch = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    with tf.device('/GPU:0'):
        # Huấn luyện mô hình qua từng epochs
        for i in range(n_epochs):
            print(i)
            # Khởi tạo batch trên tập train
            for j in range(batch_per_epoch):
                    # 1. Huấn luyện model discrinator
                    # Khởi tạo batch cho ảnh real ngẫu nhiên
                    [X_real, labels_real], y_real = _generate_real_samples(dataset, half_batch)
                    # Cập nhật discriminator model weights
                    d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
                    # Khởi tạo batch cho ảnh fake ngẫu nhiên
                    [X_fake, labels], y_fake = _generate_fake_samples(g_model, latent_dim, half_batch)
                    # Cập nhật weights cho discriminator model
                    d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
                    # 2. Huấn luyện model generator
                    # Khởi tạo các điểm ngẫu nhiên trong latent space như là đầu vào cho generator
                    [z_input, labels_input] = _generate_latent_points(latent_dim, n_batch)
                    # Khởi tạo nhãn discriminator cho các dữ liệu fake. Do chúng ta giả định là generator đánh lừa được discriminator nên nhãn của ảnh là 1.
                    y_gan = np.ones((n_batch, 1))
                    # Huấn luyện generator thông qua model cGAN
                    g_loss = cGAN_model.train_on_batch([z_input, labels_input], y_gan)

            if (i % save_every_epochs) & (i > 0):
                # summarize loss on this batch
                print('>%d, /%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, batch_per_epoch, d_loss1, d_loss2, g_loss))
                on_epoch_end(generator=g_model, num_img=3, latent_dim=latent_dim, epoch=n_epochs)
                g_model.save('cGAN_generator_epoch{}.h5'.format(i))
        # save the generator model
        g_model.save('cGAN_generator.h5')

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

# Kích thước latent space
latent_dim = 100

# Khởi tạo generator, discriminator và cGAN model
g_model = _generator()
d_model = _discriminator()
cGAN_model = _cGAN(g_model, d_model)

# load image data
dataset = _standardize_data(X_train, y_train)

In [ ]:
# train model
_train(g_model, d_model, cGAN_model, dataset, latent_dim)

0
1


>2, /468, d1=0.696, d2=0.726 g=0.771
2
